In [28]:
import pandas as pd
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score, classification_report
import warnings
warnings.filterwarnings("ignore")


In [45]:
df = pd.read_csv('with_4th_dataset.csv')

In [46]:
lb = LabelEncoder()
df['label'] = lb.fit_transform(df['label'])

In [47]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, df) -> None:
        super().__init__()
        self.y = torch.tensor(df.pop('label').values, dtype=torch.long)
        self.X = torch.unsqueeze(torch.tensor(df.iloc[:,:].values, dtype=torch.float32), dim=1)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.y)

In [48]:
init_data = MyDataset(df)
n = len(init_data)
lengths = [int(n*0.8), n-int(n*0.8)]
train_set, val_set = random_split(init_data, lengths=lengths)
len(train_set), len(val_set), n

(1199, 300, 1499)

In [49]:
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=8)
val_loader = torch.utils.data.DataLoader(train_set, shuffle=True, batch_size=8)

In [50]:
x,y = next(iter(train_loader))

In [51]:
y

tensor([1, 1, 4, 4, 1, 1, 3, 3])

In [52]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.cnn = nn.Sequential(
#             nn.Conv1d(1,1000, 7, 2),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Conv1d(1000, 500, 5, 2),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Conv1d(500, 200, 3, 2),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.MaxPool1d(7, 2),
#             nn.Conv1d(200, 100, 3, 2),
#             nn.ReLU(),
#             nn.Dropout(0.2),
#             nn.Flatten(),
#             nn.Linear(15400, 500),
#             nn.ReLU(),
#             nn.Dropout(0.1),
#             nn.Linear(500, 5),
#             nn.Softmax(dim=1)
#             )
    
#     def forward(self, x):
#         return self.cnn(x)
import torch.nn.functional as F
class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels, 
            out_channels=self.out_channels, 
            kernel_size=self.kernel_size, 
            stride=self.stride, 
            groups=self.groups)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.conv(net)

        return net
        
class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):
        
        net = x
        
        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)
        
        net = self.max_pool(net)
        
        return net
    
class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()
        
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels, 
            out_channels=out_channels, 
            kernel_size=kernel_size, 
            stride=1,
            groups=self.groups)
                
        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):
        
        identity = x
        
        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)
        
        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)
        
        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)
            
        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)
        
        # shortcut
        out += identity

        return out
    
class ResNet1D(nn.Module):
    """
    
    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)
        
    Output:
        out: (n_samples)
        
    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes
        
    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()
        
        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters
                
        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels
            
            tmp_block = BasicBlock(
                in_channels=in_channels, 
                out_channels=out_channels, 
                kernel_size=self.kernel_size, 
                stride = self.stride, 
                groups = self.groups, 
                downsample=downsample, 
                use_bn = self.use_bn, 
                use_do = self.use_do, 
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        out = x
        
        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)
        
        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)
        
        return out    
    

In [53]:
kernel_size = 16
stride = 2
n_block = 8
downsample_gap = 6
increasefilter_gap = 12
model = ResNet1D(
    in_channels=1, 
    base_filters=128, # 64 for ResNet1D, 352 for ResNeXt1D
    kernel_size=kernel_size, 
    stride=stride, 
    groups=32, 
    n_block=n_block, 
    n_classes=5, 
    # downsample_gap=downsample_gap, 
    increasefilter_gap=increasefilter_gap, 
    use_do=True)

In [44]:
# model = torch.load('model.pth')

In [59]:

EPOCHS = 100
lr = 0.0005
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

min_loss = 0.18

for i in range(EPOCHS):
    total_loss = 0
    model.train()
    model.to(device)
    for j, (data, label) in enumerate(train_loader):
        out = model(data.to(device))
        # print(out.shape)
        # print(label.shape)
        # break
        loss = loss_fn(out, label.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss
    if (total_loss/len(train_loader)) < min_loss:
      min_loss = total_loss/len(train_loader)
      name = 'model'+'_'+str(i) + '_'+str(min_loss)+'_.pth'
      torch.save(model, name)
      print('min loss changed {0}'.format(min_loss))
      
    print(i, total_loss/len(train_loader))
  

cuda:0
0 0.3170558581377069
1 0.30464287432531517
2 0.2898465188841025
3 0.292579785871009
4 0.28590234187742075
min loss changed 0.2775438614562154
5 0.2775438614562154
6 0.29409376621246336
min loss changed 0.269418256940941
7 0.269418256940941
min loss changed 0.26828449851522845
8 0.26828449851522845
9 0.27761006919046244
10 0.2759263473500808
min loss changed 0.24497292285164196
11 0.24497292285164196
12 0.2501605952406923
13 0.2576814612125357
14 0.2574148789917429
15 0.28221152156591417
16 0.2767007898228864
17 0.2451568257684509
18 0.26172968978683153
19 0.269069105759263
20 0.26455424862603344
21 0.267681183864673
22 0.2587560453886787
23 0.2499283703789115
24 0.25290167836472394
min loss changed 0.24453802927707632
25 0.24453802927707632
26 0.26264225063224633
27 0.2554334934862951
28 0.24656437678883472
min loss changed 0.23252926586816708
29 0.23252926586816708
30 0.2547215252245466
31 0.24923076532781124
32 0.2411634614566962
33 0.25516813129807514
34 0.25911834878847
35 0

In [62]:
# set model to evaluating (testing)
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
val_loss = 0
precision, recall, f1, accuracy = [], [], [], []
model.eval()
with torch.no_grad():
    for i,(data, label) in enumerate(val_loader):
        # print(data.shape)
        
        outputs = model(data.to(device))
        # print(outputs.shape)
        # break
        val_loss += loss_fn(outputs, label.to(device))
        print(val_loss/(i+1))

        predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
        
        # calculate P/R/F1/A metrics for batch
        for acc, metric in zip((precision, recall, f1), 
                                (precision_score, recall_score, f1_score)):
            acc.append(
                metric(label.cpu(), predicted_classes.cpu(), average="macro")
            )
        accuracy.append(accuracy_score(label.cpu(), predicted_classes.cpu()))

tensor(0.3708, device='cuda:0')
tensor(0.3052, device='cuda:0')
tensor(0.2536, device='cuda:0')
tensor(0.2164, device='cuda:0')
tensor(0.2006, device='cuda:0')
tensor(0.1991, device='cuda:0')
tensor(0.2346, device='cuda:0')
tensor(0.2106, device='cuda:0')
tensor(0.2023, device='cuda:0')
tensor(0.1876, device='cuda:0')
tensor(0.1751, device='cuda:0')
tensor(0.2213, device='cuda:0')
tensor(0.2049, device='cuda:0')
tensor(0.1922, device='cuda:0')
tensor(0.2264, device='cuda:0')
tensor(0.2209, device='cuda:0')
tensor(0.2105, device='cuda:0')
tensor(0.2358, device='cuda:0')
tensor(0.2251, device='cuda:0')
tensor(0.2173, device='cuda:0')
tensor(0.2140, device='cuda:0')
tensor(0.2082, device='cuda:0')
tensor(0.2009, device='cuda:0')
tensor(0.2077, device='cuda:0')
tensor(0.2113, device='cuda:0')
tensor(0.2056, device='cuda:0')
tensor(0.2020, device='cuda:0')
tensor(0.1955, device='cuda:0')
tensor(0.1906, device='cuda:0')
tensor(0.1857, device='cuda:0')
tensor(0.1813, device='cuda:0')
tensor(0

In [63]:
[sum(x*100)//len(x) for x in [precision, recall, f1, accuracy]]


[93.0, 93.0, 92.0, 95.0]

In [64]:
torch.save(model, 'model.pth')

In [ ]:
[53.0, 58.0, 52.0, 61.0] [66.0, 74.0, 68.0, 75.0] 